# Run a REopt API evaluation

## Initialization

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import copy
import os
from src.post_and_poll import get_api_results
API_KEY = 'DEMO_KEY'  # REPLACE WITH YOUR API KEY

In [2]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

In [3]:
"""
Here are some convenience definitions for using the Multi-scenario capabilities
"""
##############################################################################################################
inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')
loads_path = os.path.join(".", 'load_profiles')
rates_path = os.path.join(".", 'electric_rates')
##############################################################################################################

## Load a previously saved API response .json file instead of running REopt

In [4]:
response_json = 'results'
with open(os.path.join(outputs_path, response_json + '.json'), 'rb') as handle:
    api_response = json.load(handle)

## Scenario Inputs (POST), if wanting to do a new API call

In [5]:
post_1 = {"Scenario": {
    "Site": {
        "longitude": -118.116,
        "latitude": 34.579,
        "roof_squarefeet": 5000.0,
        "land_acres": 1.0,
    "PV": {
        "macrs_bonus_pct": 0.4,
        "installed_cost_us_dollars_per_kw": 2000.0,
        "tilt": 34.579,
        "degradation_pct": 0.005,
        "macrs_option_years": 5,
        "federal_itc_pct": 0.3,
        "module_type": 0,
        "array_type": 1,
        "om_cost_us_dollars_per_kw": 16.0,
        "macrs_itc_reduction": 0.5,
        "azimuth": 180.0,
        "federal_rebate_us_dollars_per_kw": 350.0,
        "dc_ac_ratio": 1.1
    },
    "LoadProfile": {
        "doe_reference_name": "RetailStore",
        "annual_kwh": 10000000.0,
        "city": "LosAngeles"
    },
    "Storage": {
        "total_rebate_per_kw": 100.0,
        "macrs_option_years": 5,
        "can_grid_charge": True,
        "macrs_bonus_pct": 0.4,
        "macrs_itc_reduction": 0.5,
        "total_itc_pct": 0,
        "installed_cost_us_dollars_per_kw": 1000.0,
        "installed_cost_us_dollars_per_kwh": 500.0,
        "replace_cost_us_dollars_per_kw": 460.0,
        "replace_cost_us_dollars_per_kwh": 230.0
    },
    "ElectricTariff": {
        "urdb_label": "5ed6c1a15457a3367add15ae"
    },
    "Financial": {
        "escalation_pct": 0.026,
        "offtaker_discount_pct": 0.081,
        "owner_discount_pct": 0.081,
        "analysis_years": 20,
        "offtaker_tax_pct": 0.4,
        "owner_tax_pct": 0.4,
        "om_cost_escalation_pct": 0.025
    }
}}}

### Load in a custom electric rate generated from https://reopt.nrel.gov/tool/custom_tariffs

In [6]:
load_electric_rate = "PGE_E20"
with open(os.path.join(rates_path, load_electric_rate + ".json"), 'r') as fp:
    rate_1 = json.load(fp)
post_1["Scenario"]["Site"]["ElectricTariff"]["urdb_response"] = rate_1

## Save the POST to a .json file for sharing or future loading in

In [7]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
post_name = "post_1"
with open(os.path.join(inputs_path, post_name + ".json"), 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the inputs/POST

In [4]:
# Load a json into a python dictionary
load_post = "post_ghp"
with open(os.path.join(inputs_path, load_post + ".json"), 'r') as fp:
    post_1 = json.load(fp)
post_1["Scenario"]["Site"]["GHP"]["ghpghx_inputs"][0]["ghx_fluid_thermal_conductivity_btu_per_hr_ft_f"] = 0.34
post_1["Scenario"]["Site"]["GHP"]["ghpghx_inputs"][0]["ghx_fluid_dynamic_viscosity_lbm_per_ft_hr"] = 2.75

In [5]:
post_1["Scenario"]["Site"]["GHP"]

{'installed_cost_heatpump_us_dollars_per_ton': '1.0',
 'installed_cost_ghx_us_dollars_per_ft': '0.01',
 'building_sqft': '150000.0',
 'require_ghp_purchase': True,
 'macrs_option_years': 5,
 'macrs_bonus_pct': '1.0',
 'ghpghx_inputs': [{'ghx_fluid_thermal_conductivity_btu_per_hr_ft_f': 0.34,
   'ghx_fluid_dynamic_viscosity_lbm_per_ft_hr': 2.75}]}

## POST and poll (periodic GET request) the API to GET a new result, if not loading in a previous response. This may take a while!

### Note, the `api_url` in the `get_api_results` function below calls the **production server** hosted API (master/main branch/version, publicly accessible)

#### For calling a locally-hosted (localhost) API, see:
- https://github.com/NREL/REopt_Lite_API/wiki/localhost-URLs-for-calling-locally-hosted-API

#### For calling an API hosted on an NREL-internal server (only NREL users can access this), see:
- https://github.nrel.gov/REopt/API_scripts/wiki/API-URLs-for-NREL-internal-servers

`get_api_results` POST's your inputs to the API `job` endpoint, which provides a `run_uuid` if the input is valid, and then polls the `results` endpoint using the `run_uuid` until the results come back with a status other than `Optimizing...`.

`get_api_results` also saves the results (full API response, including inputs) to the `results_file`.

A log file is also created in the current working directory.

In [6]:
outputs_file_name = "results_file"
root_url = "https://developer.nrel.gov/api/reopt/v1"
api_response = get_api_results(post=post_1, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file=os.path.join(outputs_path, outputs_file_name + ".json"), 
                               run_id=None)

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=DEMO_KEY.
main         INFO     Polling https://developer.nrel.gov/api/reopt/v1/job/54b133f4-c095-4c24-b3d6-e7edf4e76c48/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to ./outputs/results_file.json


### If you get disconnected from the polling function but you think it ran, copy the run_uuid from the log above to manually GET the results:

In [4]:
run_uuid = "615ccdfb-e8b9-4f7b-bfcf-73d11ce2e2a6" #api_response["outputs"]["Scenario"]["run_uuid"]
root_url = "https://developer.nrel.gov/api/reopt/v1"
results_url = root_url + '/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

## Get summary of results

In [5]:
print("NPV ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["npv_us_dollars"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotTES", "ColdTES", "AbsorptionChiller", "GHP", "NewBoiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1["Scenario"]["Site"].keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        # PV and Storage are considered if the POST does not explicitly make max_[size] == 0
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])
    elif tech in ["PV", "Storage"]:
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])                

NPV ($) =  None
Capital Cost, Net ($) =  None


NameError: name 'post_1' is not defined

### Here are some results keys examples:

In [6]:
api_response.keys()

dict_keys(['outputs', 'inputs', 'messages'])

In [7]:
api_response["outputs"]["Scenario"]["status"]

'An error occurred. See messages for more.'

In [11]:
api_response["messages"]

{'warnings': "{'Default values used for the following:': {'Scenario': 'timeout_seconds AND time_steps_per_hour AND optimality_tolerance_bau AND add_soc_incentive', 'Scenario>Site': 'outdoor_air_temp_degF AND elevation_ft', 'Scenario>Site>Financial': 'om_cost_escalation_pct AND escalation_pct AND generator_fuel_escalation_pct AND boiler_fuel_escalation_pct AND chp_fuel_escalation_pct AND newboiler_fuel_escalation_pct AND offtaker_tax_pct AND offtaker_discount_pct AND third_party_ownership AND owner_tax_pct AND owner_discount_pct AND analysis_years AND value_of_lost_load_us_dollars_per_kwh AND microgrid_upgrade_cost_pct AND other_capital_costs_us_dollars AND other_annual_costs_us_dollars_per_year AND co2_cost_us_dollars_per_tonne AND co2_cost_escalation_pct AND owner_discount_pct AND owner_tax_pct', 'Scenario>Site>LoadProfile': 'percent_share AND year AND loads_kw_is_net AND critical_loads_kw_is_net AND critical_load_pct AND outage_is_major_event AND min_load_met_pct AND sr_required_pct'

In [16]:
for k in api_response["outputs"]["Scenario"]["Site"].keys():
    print(k)

year_one_emissions_lb_C02
year_one_emissions_bau_lb_C02
outdoor_air_temp_degF
renewable_electricity_energy_pct
Financial
LoadProfile
LoadProfileBoilerFuel
LoadProfileChillerThermal
ElectricTariff
FuelTariff
Storage
Generator
Wind
CHP
Boiler
ElectricChiller
AbsorptionChiller
HotTES
ColdTES
NewBoiler
SteamTurbine
GHP
PV


## Save API response into a JSON file for later use

In [17]:
response_save = api_response
file_name_to_save = "response_1"
with open(os.path.join(outputs_path, file_name_to_save + ".json"), 'w') as fp:
    json.dump(response_save, fp)